# Installs

In [ ]:
pip install soccer-twos

In [ ]:
pip install protobuf==3.20

In [ ]:
!pip install gym==0.19.0
!pip install gym-unity==0.27.0
!pip install mlagents==0.27.0
!pip install mlagents-envs==0.27.0

# Imports e utils

In [ ]:
import torch
import gym
import ray
from ray import tune
from soccer_twos import EnvType
import numpy as np
import os
from ray.rllib import MultiAgentEnv
from ray.rllib.agents.ppo import ppo
from ray.tune.logger import pretty_print
import soccer_twos
import collections
import random
from collections import deque
from gym_unity.envs import ActionFlattener
import numpy as np
from tqdm import tqdm

In [ ]:
class RLLibWrapper(gym.core.Wrapper, MultiAgentEnv):
    """
    A RLLib wrapper so our env can inherit from MultiAgentEnv.
    """

    pass


def create_rllib_env(env_config: dict = {}):
    """
    Creates a RLLib environment and prepares it to be instantiated by Ray workers.
    Args:
        env_config: configuration for the environment.
            You may specify the following keys:
            - variation: one of soccer_twos.EnvType. Defaults to EnvType.multiagent_player.
            - opponent_policy: a Callable for your agent to train against. Defaults to a random policy.
    """
    if hasattr(env_config, "worker_index"):
        env_config["worker_id"] = (
            env_config.worker_index * env_config.get("num_envs_per_worker", 1)
            + env_config.vector_index
        )
    env = soccer_twos.make(**env_config)
    if "multiagent" in env_config and not env_config["multiagent"]:
        # is multiagent by default, is only disabled if explicitly set to False
        return env
    return RLLibWrapper(env)

In [ ]:
tune.registry.register_env("Soccer", create_rllib_env)
temp_env = create_rllib_env({"variation": EnvType.multiagent_player})
obs_space = temp_env.observation_space
act_space = temp_env.action_space
temp_env.close()

In [ ]:
def map_policy(agent_id):
    if agent_id == 0:
        return "policy_01" # Choose 01 policy for agent_01
    else:
        return np.random.choice(["policy_01", "policy_02", "policy_03", "policy_04"],1,
                                p=[.8, .2/3, .2/3, .2/3])[0]

# Treinamento

In [ ]:
config = ppo.DEFAULT_CONFIG.copy()
checkpoint = 'resultados/ppo_1'
confing_params = {
        "num_gpus": 1,
        "num_workers": 8,
        "num_envs_per_worker": 3,
        "log_level": "INFO",
        "framework": "torch",
        "ignore_worker_failures": True,
        "train_batch_size": 256,
        "lr": 3e-4,
        "lambda": .95,
        "gamma": .998,
        "entropy_coeff": 0.01,
        "kl_coeff": 1.0,
        "clip_param": 0.2,
        "num_sgd_iter": 10,
        "observation_filter": "NoFilter",  
        "vf_loss_coeff": 1e-4,    
                               
        "vf_clip_param": 100.0,
        "multiagent": {
            "policies": {
                "policy_01": (None, obs_space, act_space, {}),
                "policy_02": (None, obs_space, act_space, {}),
                "policy_03": (None, obs_space, act_space, {}),
                "policy_04": (None, obs_space, act_space, {})
            },
            "policy_mapping_fn": map_policy,
            
        },
        "env": "Soccer",
        "env_config": {
            "num_envs_per_worker": NUM_ENVS_PER_WORKER,
            "variation": EnvType.multiagent_player,
        },
    }

config.update(config_params)
trainer = ppo.PPOTrainer(
    env="Soccer",
    config=config
)

In [ ]:
resultados = []
for i in range(int(1e3)): #1000 its
    resultados.append(trainer.train())
    print(pretty_print(trainer.train()))

    if i % 5 == 0:
        trainer.set_weights({"policy_04": trainer.get_weights(["policy_03"])["policy_03"],
                            "policy_03": trainer.get_weights(["policy_02"])["policy_02"],
                            "policy_02": trainer.get_weights(["policy_01"])["policy_01"],
                            })

    if i % 100 == 0:
        if not os.path.exists(checkpoint):
            os.makedirs(checkpoint)
        trainer.save_checkpoint(checkpoint)

In [16]:
print(resultados[0])#verificando se estava gerando certinho

{'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'policy_04': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.9134443521499633, 'cur_kl_coeff': 1.0, 'cur_lr': 0.0003, 'total_loss': -0.07702102474868297, 'policy_loss': -0.0495678664100442, 'vf_loss': 3.0617728816650926e-05, 'vf_explained_var': -0.6097293734550476, 'kl': 0.005451017414452508, 'entropy': 3.2904178380966185, 'entropy_coeff': 0.009999999999999998}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 44.0, 'num_grad_updates_lifetime': 5.5, 'diff_num_grad_updates_vs_sampler_policy': 4.5}, 'policy_01': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 0.5213890782424382, 'cur_kl_coeff': 1.0, 'cur_lr': 0.00030000000000000003, 'total_loss': -0.06548989088873246, 'policy_loss': -0.03869665548804083, 'vf_loss': 3.560547449004454e-06, 'vf_explained_var': 0.10222969821521215, 'kl': 0.0061039048382308, 'entropy': 3.2897141524723597, 'entropy_coeff': 0.01}, 'model': {}, 'custom_metrics':